### Importing Libraries

In [ ]:
import numpy as np
import time
import cv2
import os
import glob
from moviepy.editor import *

### Load COCO Dataset

In [ ]:
labels = open('coco.names').read().strip().split('\n')


### Load Neural Netowrk

In [ ]:
net = cv2.dnn.readNetFromDarknet('yolov3.cfg','yolov3.weights')

### Getting Layer Names and Output Layers

In [ ]:
names = net.getLayerNames()
outputlayers = list(net.getUnconnectedOutLayersNames())  


### Inference

In [ ]:
def process(image):
    boxes = []
    confidences = []
    classIDs = []
    H,W = image.shape[:2]   #height and width of the image
    blob = cv2.dnn.blobFromImage(image,1/255.0,(416,416),crop=False,swapRB=False)  #preprocessing the image
    net.setInput(blob)   #input the image to the network
    layers_output = net.forward(outputlayers)  #forward path
    
    for output in layers_output:
        for detection in output:
            scores = detection[5:]   #getting the scores of all the classes           
            classID = np.argmax(scores)  #getting ID of the class that has maximum score
            confidence = scores[classID]  #confidence of the class
            
            if confidence > 0.75:
                box = detection[:4] * np.array([W,H,W,H])  #Getting center,width and height of the box
                bx,by,bw,bh = box.astype("int")   
                x = int(bx-(bw/2))  # x-coordinate of top left corner point
                y = int(by-(bh/2))  # y-coordinate of top left corner point
                
                boxes.append([x,y,int(bw),int(bh)])  #inserting the box coordinates in boxes
                confidences.append(float(confidence)) #inserting confidence of the class in confidences
                classIDs.append(classID)  #inserting class Index in ClassIDs
                
                
   
    return image
